In [1]:
# hide
# all_slow

In [2]:
# hide
# no_test
![ -e /content ] && pip install -Uqq pyarrow fastparquet
![ -e /content ] && pip install -Uqq geowrangler

# Datasets Download
> Basic introduction to downloading open geospatial data using geowrangler

In [3]:
%reload_ext autoreload
%autoreload 2

## Downloading Geofabrik Data

In [4]:
import geopandas as gpd

from geowrangler.datasets import geofabrik

### Listing down available regions

In [5]:
regions = geofabrik.list_geofabrik_regions()
# list down regions in asia
{k: v for k, v in regions.items() if "asia" in v}

{'afghanistan': 'https://download.geofabrik.de/asia/afghanistan-latest-free.shp.zip',
 'armenia': 'https://download.geofabrik.de/asia/armenia-latest-free.shp.zip',
 'azerbaijan': 'https://download.geofabrik.de/asia/azerbaijan-latest-free.shp.zip',
 'bangladesh': 'https://download.geofabrik.de/asia/bangladesh-latest-free.shp.zip',
 'bhutan': 'https://download.geofabrik.de/asia/bhutan-latest-free.shp.zip',
 'cambodia': 'https://download.geofabrik.de/asia/cambodia-latest-free.shp.zip',
 'central-zone': 'https://download.geofabrik.de/asia/india/central-zone-latest-free.shp.zip',
 'china': 'https://download.geofabrik.de/asia/china-latest-free.shp.zip',
 'chubu': 'https://download.geofabrik.de/asia/japan/chubu-latest-free.shp.zip',
 'chugoku': 'https://download.geofabrik.de/asia/japan/chugoku-latest-free.shp.zip',
 'east-timor': 'https://download.geofabrik.de/asia/east-timor-latest-free.shp.zip',
 'eastern-zone': 'https://download.geofabrik.de/asia/india/eastern-zone-latest-free.shp.zip',
 '

### Downloading a region file to a directory


In [6]:
# hide
# no_test
!rm -r ../test_dir
!mkdir -p ../test_dir

In [7]:
%%time
# no_test
downloaded_file = geofabrik.download_geofabrik_region("laos", "../test_dir")
downloaded_file

2023-02-01 17:00:04.495 | INFO     | geowrangler.datasets.utils:urlretrieve:27 - Retrieving https://download.geofabrik.de/asia/laos-latest-free.shp.zip into ../test_dir/laos-latest-free.shp.zip


CPU times: user 1.58 s, sys: 980 ms, total: 2.56 s
Wall time: 18 s


Path('../test_dir/laos-latest-free.shp.zip')

### Loading geofabrik files

In [8]:
%%time
gdf = gpd.read_file(downloaded_file)
gdf.head()

CPU times: user 19.4 s, sys: 528 ms, total: 19.9 s
Wall time: 19.9 s


,osm_id,code,fclass,name,type,geometry
0,36192811,1500,building,Don Chan Palace,None,"POLYGON ((102.61392 17.95514, 102.61419 17.955..."
1,36195671,1500,building,ຕະຫລາດເຊົ້າ. 早市塲,None,"POLYGON ((102.61382 17.96520, 102.61393 17.965..."
2,36195973,1500,building,Mercure Hotel Laotel,None,"POLYGON ((102.59386 17.96890, 102.59432 17.968..."
3,36784437,1500,building,None,None,"POLYGON ((105.79786 15.11905, 105.79838 15.119..."
4,37886432,1500,building,Presidential Palace,None,"POLYGON ((102.60940 17.96246, 102.60951 17.962..."


You can also list the contents of the zipped shape file as well as load the shape file within it

In [9]:
!unzip -l {downloaded_file.as_posix()}

Archive:  ../test_dir/laos-latest-free.shp.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
      647  2023-02-01 09:00   README
        6  2023-02-01 09:00   gis_osm_buildings_a_free_1.cpg
 68322734  2023-02-01 09:00   gis_osm_buildings_a_free_1.dbf
      144  2023-02-01 09:00   gis_osm_buildings_a_free_1.prj
 60089724  2023-02-01 09:00   gis_osm_buildings_a_free_1.shp
  3312708  2023-02-01 09:00   gis_osm_buildings_a_free_1.shx
        6  2023-02-01 09:00   gis_osm_landuse_a_free_1.cpg
  2845642  2023-02-01 09:00   gis_osm_landuse_a_free_1.dbf
      144  2023-02-01 09:00   gis_osm_landuse_a_free_1.prj
 14407732  2023-02-01 09:00   gis_osm_landuse_a_free_1.shp
   157092  2023-02-01 09:00   gis_osm_landuse_a_free_1.shx
        6  2023-02-01 09:00   gis_osm_natural_a_free_1.cpg
     7702  2023-02-01 09:00   gis_osm_natural_a_free_1.dbf
      144  2023-02-01 09:00   gis_osm_natural_a_free_1.prj
    15556  2023-02-01 09:00   gis_osm_natural_a_free_1.shp
      516 

In [10]:
%%time
gdf2 = gpd.read_file(f"{downloaded_file.as_posix()}!gis_osm_pois_free_1.shp")

CPU times: user 745 ms, sys: 19.4 ms, total: 764 ms
Wall time: 773 ms


### Unzipping and Caching OSM 

In addition downloading, the geofabrik module provides an unzipping and caching facility (default cache directory: `~/.cache/geowrangler/osm` ) to make it easier to access OSM data.

In [11]:
%%time
download_path = geofabrik.download_osm_region_data("afghanistan")
download_path

2023-02-01 17:00:42.767 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:127 - OSM Data: Cached data available for afghanistan at /home/butchtm/.cache/geowrangler/osm/afghanistan-latest-free.shp.zip? False
2023-02-01 17:00:43.526 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:141 - OSM Data: Re-initializing OSM region zip file at /home/butchtm/.cache/geowrangler/osm/afghanistan-latest-free.shp.zip...
2023-02-01 17:00:43.529 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:148 - OSM Data: Downloading Geofabrik in /home/butchtm/.cache/geowrangler/osm/afghanistan-latest-free.shp.zip...
2023-02-01 17:00:44.247 | INFO     | geowrangler.datasets.utils:urlretrieve:27 - Retrieving https://download.geofabrik.de/asia/afghanistan-latest-free.shp.zip into /home/butchtm/.cache/geowrangler/osm/afghanistan-latest-free.shp.zip


2023-02-01 17:01:23.951 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:159 - OSM Data: Successfully downloaded and cached OSM data for afghanistan at /home/butchtm/.cache/geowrangler/osm/afghanistan-latest-free.shp.zip!


CPU times: user 3.22 s, sys: 1.76 s, total: 4.98 s
Wall time: 41.2 s


Path('/home/butchtm/.cache/geowrangler/osm/afghanistan-latest-free.shp.zip')

Downloading it the second time around should be much faster as it
only checks if its in the cache and returns the path

In [12]:
%%time
download_path2 = geofabrik.download_osm_region_data("afghanistan")
download_path2

2023-02-01 17:01:23.973 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:127 - OSM Data: Cached data available for afghanistan at /home/butchtm/.cache/geowrangler/osm/afghanistan-latest-free.shp.zip? True


CPU times: user 2.22 ms, sys: 349 µs, total: 2.57 ms
Wall time: 2.08 ms


Path('/home/butchtm/.cache/geowrangler/osm/afghanistan-latest-free.shp.zip')

In [25]:
!ls -aldh  {download_path2}

-rw-r--r-- 1 butchtm butchtm 231M Feb  1 17:01 /home/butchtm/.cache/geowrangler/osm/afghanistan-latest-free.shp.zip


### Using the OSM Data Manager

We also provide an OSM Data Manager which, in addition to using the default cache (`~/.cache/geowrangler/osm`),
also caches the geodataframe for either the pois or the roads datasets from OSM in memory to avoid having
to reload the OSM data from disk.

In [14]:
# Create the osm data manager
odm = geofabrik.OsmDataManager()

In [15]:
pois_ph = odm.load_pois("philippines")

2023-02-01 17:01:24.214 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:127 - OSM Data: Cached data available for philippines at /home/butchtm/.cache/geowrangler/osm/philippines-latest-free.shp.zip? False
2023-02-01 17:01:25.631 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:141 - OSM Data: Re-initializing OSM region zip file at /home/butchtm/.cache/geowrangler/osm/philippines-latest-free.shp.zip...
2023-02-01 17:01:25.634 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:148 - OSM Data: Downloading Geofabrik in /home/butchtm/.cache/geowrangler/osm/philippines-latest-free.shp.zip...
2023-02-01 17:01:26.355 | INFO     | geowrangler.datasets.utils:urlretrieve:27 - Retrieving https://download.geofabrik.de/asia/philippines-latest-free.shp.zip into /home/butchtm/.cache/geowrangler/osm/philippines-latest-free.shp.zip


2023-02-01 17:09:06.368 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:159 - OSM Data: Successfully downloaded and cached OSM data for philippines at /home/butchtm/.cache/geowrangler/osm/philippines-latest-free.shp.zip!
2023-02-01 17:09:06.370 | DEBUG    | geowrangler.datasets.geofabrik:load_pois:218 - OSM POIs for philippines being loaded from /home/butchtm/.cache/geowrangler/osm/philippines-latest-free.shp.zip


In [16]:
pois_ph.head()

,osm_id,code,fclass,name,geometry
0,21717820,2907,camera_surveillance,None,POINT (121.02120 14.57608)
1,21717872,2722,museum,Ayala Museum,POINT (121.02324 14.55358)
2,24078301,2402,motel,Sogo Grand Hotel,POINT (121.04515 14.56449)
3,24078959,2907,camera_surveillance,None,POINT (121.05945 14.60098)
4,24797511,2542,bicycle_shop,Christine Sports Cycle Marketing,POINT (120.99506 14.55224)


In [17]:
roads_ph = odm.load_roads("philippines")

2023-02-01 17:09:10.649 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:127 - OSM Data: Cached data available for philippines at /home/butchtm/.cache/geowrangler/osm/philippines-latest-free.shp.zip? True
2023-02-01 17:09:10.653 | DEBUG    | geowrangler.datasets.geofabrik:load_roads:274 - OSM Roads for philippines being loaded from /home/butchtm/.cache/geowrangler/osm/philippines-latest-free.shp.zip


In [18]:
roads_ph.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,267,5115,tertiary,Maharlika Street,None,F,30,0,F,F,"LINESTRING (121.05195 14.65011, 121.05209 14.6..."
1,2508359,5153,footway,Sagada Mission Compound,None,B,0,0,F,F,"LINESTRING (120.90199 17.08090, 120.90197 17.0..."
2,2667097,5113,primary,Taft Avenue,170,F,60,0,F,F,"LINESTRING (120.99682 14.55577, 120.99671 14.5..."
3,2667099,5113,primary,Lerma Street,170,F,60,0,F,F,"LINESTRING (120.98539 14.60460, 120.98545 14.6..."
4,2667105,5122,residential,Manalito Street,None,B,0,0,F,F,"LINESTRING (120.99380 14.54125, 120.99497 14.5..."


## Loading OSM data from other years

In addition to providing access to the latest OSM shape files, we also provide an optional `year` parameter, which allows you to download OSM data from previous years. 

> Note: The availability of data from previous years is dependent on what geofabrik has made available. Please check the [Geofabrik download site](https://download.geofabrik.de/) for the list of available data 

In [19]:
pois_ph = odm.load_pois("philippines", year="2017")

2023-02-01 17:10:30.351 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:127 - OSM Data: Cached data available for philippines at /home/butchtm/.cache/geowrangler/osm/philippines-170101-free.shp.zip? False
2023-02-01 17:10:33.834 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:141 - OSM Data: Re-initializing OSM region zip file at /home/butchtm/.cache/geowrangler/osm/philippines-170101-free.shp.zip...
2023-02-01 17:10:33.838 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:148 - OSM Data: Downloading Geofabrik in /home/butchtm/.cache/geowrangler/osm/philippines-170101-free.shp.zip...
2023-02-01 17:10:35.780 | INFO     | geowrangler.datasets.utils:urlretrieve:27 - Retrieving https://download.geofabrik.de/asia/philippines-170101-free.shp.zip into /home/butchtm/.cache/geowrangler/osm/philippines-170101-free.shp.zip


2023-02-01 17:12:24.933 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:163 - OSM Data: Successfully downloaded and cached OSM data for philippines and 2017 at /home/butchtm/.cache/geowrangler/osm/philippines-170101-free.shp.zip!
2023-02-01 17:12:24.935 | DEBUG    | geowrangler.datasets.geofabrik:load_pois:220 - OSM POIs for philippines and year 2017 being loaded from /home/butchtm/.cache/geowrangler/osm/philippines-170101-free.shp.zip


In [20]:
pois_ph

,osm_id,code,fclass,name,geometry
0,14401658,2742,viewpoint,East Point,POINT (120.62020 14.38562)
1,14446500,2721,attraction,Intramuros,POINT (120.97533 14.59059)
2,21717872,2722,museum,Ayala Museum,POINT (121.02324 14.55358)
3,24078301,2401,hotel,Sogo Grand Hotel,POINT (121.04515 14.56449)
4,24078959,2907,camera_surveillance,None,POINT (121.05945 14.60098)
...,...,...,...,...,...
67400,4584129489,2502,bakery,Come-on bakery,POINT (123.26424 9.19066)
67401,4584130190,2401,hotel,L-mansion,POINT (121.00280 14.55968)
67402,4584164592,2721,attraction,Lugnason Falls,POINT (123.53406 9.14808)
67403,4584215391,2742,viewpoint,None,POINT (123.53406 9.14804)


## Downloading Ookla Data


In [21]:
import geopandas as gpd
import pandas as pd

from geowrangler.datasets import ookla

### Listing down available files

In [22]:
ookla_files = ookla.list_ookla_files()
ookla_files

{OoklaQuarter(type='fixed', year='2019', quarter='1'): '2019-01-01_performance_fixed_tiles.parquet',
 OoklaQuarter(type='fixed', year='2019', quarter='2'): '2019-04-01_performance_fixed_tiles.parquet',
 OoklaQuarter(type='fixed', year='2019', quarter='3'): '2019-07-01_performance_fixed_tiles.parquet',
 OoklaQuarter(type='fixed', year='2019', quarter='4'): '2019-10-01_performance_fixed_tiles.parquet',
 OoklaQuarter(type='fixed', year='2020', quarter='1'): '2020-01-01_performance_fixed_tiles.parquet',
 OoklaQuarter(type='fixed', year='2020', quarter='2'): '2020-04-01_performance_fixed_tiles.parquet',
 OoklaQuarter(type='fixed', year='2020', quarter='3'): '2020-07-01_performance_fixed_tiles.parquet',
 OoklaQuarter(type='fixed', year='2020', quarter='4'): '2020-10-01_performance_fixed_tiles.parquet',
 OoklaQuarter(type='fixed', year='2021', quarter='1'): '2021-01-01_performance_fixed_tiles.parquet',
 OoklaQuarter(type='fixed', year='2021', quarter='2'): '2021-04-01_performance_fixed_tiles.

### Downloading an ookla file to a directory
> Warning: Ookla files are >300MB and can reach ~550MB. Download with caution

In [23]:
!mkdir -p ../test_dir
downloaded_file = ookla.download_ookla_file(
    type_="fixed", year="2019", quarter="1", directory="../test_dir"
)
downloaded_file

Path('../test_dir/2019-01-01_performance_fixed_tiles.parquet')

### Loading ookla data

In [24]:
df = pd.read_parquet(downloaded_file)
df.head()

,quadkey,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices
0,0022133222312322,"POLYGON((-160.02685546875 70.6435894914449, -1...",8763,3646,45,1,1
1,0022133222330013,"POLYGON((-160.032348632812 70.6399478155463, -...",9195,3347,43,1,1
2,0022133222330023,"POLYGON((-160.043334960938 70.6363054807905, -...",6833,3788,42,1,1
3,0022133222330100,"POLYGON((-160.02685546875 70.6417687358462, -1...",8895,3429,43,2,2
4,0022320121121332,"POLYGON((-166.739501953125 68.3526207780586, -...",4877,935,45,3,2
